In [22]:
# moduel
from my_package.select_dataset_all import get_all_dataframe_from_database
import my_package.time_series as time

# basic
import pandas as pd
import numpy as np
import pickle

# visualize
import matplotlib.pyplot as plt  

# learning
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

<span style="color: blue; font-size: 14px; font-weight: bold; background-color: #f0f0f0; padding: 5px; border-radius: 5px;">
    set
</span>

In [2]:
# goals: 데이터 생성 

data = time.create_electrode_data()

In [3]:
# goals: 데이터 처리

ship_name = 'MASAN TEST150'

ship_id = ship_id = time.find_ship(ship_name)[0]

df = time.preprocess_data(data,ship_id)

df = time.generate_moving_average(df)

In [4]:
# golas: 시간 기준 그룹화

group = df.groupby(['DATA_TIME']).mean()['ELECTRODE_EFFICIENCY'].to_frame().reset_index()

C:\Users\pc021\AppData\Local\Temp\ipykernel_12208\2710879781.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group = df.groupby(['DATA_TIME']).mean()['ELECTRODE_EFFICIENCY'].to_frame().reset_index()


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# 데이터 로드
# df = pd.read_csv('path_to_your_data.csv')  # 실제 데이터 경로를 입력해 주세요.
# 위 데이터를 예로 사용한 더미 데이터 생성

# 1. 데이터를 ELECTRODE_EFFICIENCY 값만 추출하여 numpy array로 변환
data = group['ELECTRODE_EFFICIENCY'].values.reshape(-1, 1)  # (74581, 1) 형태

# 2. 학습용 데이터 준비: 시퀀스를 나누어 학습과 예측에 사용
sequence_length = 3000  # 한 번에 입력할 시퀀스 길이
future_length = 300  # 예측할 미래 시퀀스 길이

# 입력 시퀀스를 만들기 위한 함수
def create_sequences(data, sequence_length, future_length):
    X, y = [], []
    for i in range(len(data) - sequence_length - future_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length:i + sequence_length + future_length])
    return np.array(X), np.array(y)

# X_train: 시퀀스 입력, y_train: 미래 예측값
X_train, y_train = create_sequences(data, sequence_length, future_length)

# 3. Transformer 모델 구성
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(embed_dim, activation="relu"),
            tf.keras.layers.Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.add = tf.keras.layers.Add()

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        out1 = self.add([inputs, attn_output])  # Skip connection
        out1 = self.layernorm1(out1)  # Normalization after attention
        ffn_output = self.ffn(out1)
        ffn_output = self.layernorm2(self.add([out1, ffn_output]))  # Skip connection
        return ffn_output

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim):
        super(PositionalEncoding, self).__init__()
        self.positional_encoding = self.positional_encoding_matrix(sequence_length, embed_dim)

    def positional_encoding_matrix(self, sequence_length, embed_dim):
        angle_rads = self.get_angles(np.arange(sequence_length)[:, np.newaxis],
                                     np.arange(embed_dim)[np.newaxis, :],
                                     embed_dim)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # apply sin to even indices
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # apply cos to odd indices
        return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

    def get_angles(self, position, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return position * angle_rates

    def call(self, inputs):
        return inputs + self.positional_encoding[:, :tf.shape(inputs)[1], :]

# 입력 시퀀스 형태 정의
sequence_length = 3000
embed_dim = 16  # 임베딩 차원
num_heads = 2   # 멀티헤드 어텐션의 헤드 수

inputs = tf.keras.layers.Input(shape=(sequence_length, 1))  # (3000, 1)

# Positional Encoding 적용
pos_encoding = PositionalEncoding(sequence_length, embed_dim)(inputs)

# 임베딩 차원으로 변환
embedded_inputs = tf.keras.layers.Dense(embed_dim)(inputs)

# Transformer 블록 적용
transformer_block = TransformerBlock(embed_dim, num_heads)(embedded_inputs)

# 예측할 미래 값(300분)
outputs = tf.keras.layers.Dense(1)(transformer_block[:, -future_length:, :])

# 모델 생성
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse')

# 4. 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32)

# 5. 예측 (새로운 입력 데이터로 예측)
y_pred = model.predict(X_train[-1].reshape(1, sequence_length, 1))  # 마지막 시퀀스로 예측
print("Predicted values for the next 300 minutes:", y_pred)


Epoch 1/10
  25/2228 [..............................] - ETA: 2:34:01 - loss: 1156.0520